In [1]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal


In [2]:
import aiida
aiida.load_profile()


In [3]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


In [4]:
# nodebank settings

from tools.nodebank import NodeBank
pwd = os.getcwd()
print(pwd)
nodebank = NodeBank(pwd)


/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA


In [5]:
!pwd

/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA


In [6]:
import os
import numpy as np


from aiida.plugins import DataFactory
from aiida.orm import Code
from aiida.orm import Str, Dict, List, Int
from aiida.engine import calcfunction, WorkChain

from CrySPY.IO import read_input as rin
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')

# PandasFrameData = DataFactory('cryspy.dataframe')
PandasFrameData = DataFactory('dataframe.frame')

ConfigparserData = DataFactory('cryspy.configparser')
StructurecollectionData = DataFactory('cryspy.structurecollection')
EAData = DataFactory('cryspy.ea_data')
EAidData = DataFactory('cryspy.ea_id_data')


In [7]:
pwd = os.getcwd()
cryspy_in = os.path.join(pwd, 'cryspy.in0')
inputs = {'cryspy_in': Str(cryspy_in)}
initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
initial_result = run(initialize_WorkChain, **inputs)


2022/09/14 07:35:28
CrySPY 0.10.3
Start cryspy.py

Read input file, /home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/cryspy.in0
Write input data in cryspy.out
Save input data in cryspy.stat

# --------- Generate initial structures
Structure ID      0 was generated. Space group:  79 -->  79 I4
Structure ID      1 was generated. Space group:  56 -->  56 Pccn
Structure ID      2 was generated. Space group:  55 -->  65 Cmmm
Structure ID      3 was generated. Space group:   4 -->   4 P2_1


Compoisition [16 16] not compatible with symmetry 209 :spg = 209 retry.
Compoisition [4 4] not compatible with symmetry 153 :spg = 153 retry.


Structure ID      4 was generated. Space group:  67 -->  67 Cmme
Structure ID      5 was generated. Space group: 107 --> 107 I4mm
Structure ID      6 was generated. Space group:   5 -->   5 C2
Structure ID      7 was generated. Space group: 126 --> 139 I4/mmm
Structure ID      8 was generated. Space group:  13 -->  13 P2/c
Structure ID      9 was generated. Space group: 122 --> 122 I-42d


Compoisition [4 4] not compatible with symmetry 169 :spg = 169 retry.
Compoisition [8 8] not compatible with symmetry 220 :spg = 220 retry.
Compoisition [16 16] not compatible with symmetry 219 :spg = 219 retry.
Compoisition [4 4] not compatible with symmetry 144 :spg = 144 retry.


Structure ID     10 was generated. Space group: 166 --> 166 R-3m
Structure ID     11 was generated. Space group:  55 -->  55 Pbam
Structure ID     12 was generated. Space group: 205 --> 225 Fm-3m
Structure ID     13 was generated. Space group: 134 --> 134 P4_2/nnm
Structure ID     14 was generated. Space group:  62 -->  62 Pnma
Structure ID     15 was generated. Space group:  77 -->  77 P4_2
Structure ID     16 was generated. Space group: 158 --> 156 P3m1
Structure ID     17 was generated. Space group: 147 --> 191 P6/mmm
Structure ID     18 was generated. Space group:  37 -->  37 Ccc2
Structure ID     19 was generated. Space group:  71 -->  71 Immm
Structure ID     20 was generated. Space group:  74 -->  74 Imma
Structure ID     21 was generated. Space group:   7 -->   7 Pc
Structure ID     22 was generated. Space group: 168 --> 183 P6mm
Structure ID     23 was generated. Space group:  93 --> 131 P4_2/mmc
Structure ID     24 was generated. Space group:  15 -->  15 C2/c
Structure ID    

In [8]:
# workflow結果表示
initial_result


{'init_struc': <StructureCollectionData: uuid: 9481ab16-109d-498f-a606-87acf8ff9b53 (pk: 32277)>,
 'rslt_data': <PandasFrameData: uuid: 558f9e45-6ecb-45d1-8c60-666f226e1fd5 (pk: 32278)>,
 'ea_id_data': <EAidData: uuid: 029ce993-0c14-4139-88ab-45069eb3d32f (pk: 32279)>,
 'ea_data': <EAData: uuid: 9b12f250-ecb2-4cad-ab9f-bb77344c24c9 (pk: 32280)>,
 'stat': <ConfigParserData: uuid: b59f19bb-a891-45dd-96bd-c48c06149eea (pk: 32281)>}

In [9]:
stat = initial_result["stat"]


In [10]:
stat.configparser["structure"]["symprec"]

'0.01'

In [11]:
rslt_data_node = initial_result["rslt_data"]


In [12]:
import shutil
# cryspy.inは実行中に書き換えられる。
shutil.copy("cryspy.in0", "cryspy.in")


'cryspy.in'

# 

In [13]:
# all the initial structures

all_initial_structures_node = initial_result["init_struc"]


# select structures to optimize

In [14]:
inputs = {"ea_id_data":  initial_result["ea_id_data"], "init_struc":initial_result["init_struc"] }

select_structure_to_run_WorkChain = WorkflowFactory('cryspy.select_structures')
select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["init_struc"]


In [15]:
ea_id_node = select_structure_result["ea_id_data"]


In [16]:
computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label


'lammps.optimize-lammps-localhost'

In [17]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes


{'potential_type': 'tersoff',
 'atom_style': 'atomic',
 'default_units': 'metal',
 'allowed_element_names': ['Ga', 'N'],
 'md5|input_lines': '175c64eb9b54b8238b4b32673833f366',
 'md5|potential_pot': 'b3b7d45ae7b92eba05ed99ffe69810d0',
 'external_files': ['potential.pot']}

In [18]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [19]:
code_lammps_opt


<Code: Remote code 'lammps.optimize-lammps-localhost' on localhost, pk: 1559, uuid: 5f9dad43-2953-4d9a-bced-caaea2e212d2>

In [20]:
if len(all_initial_structures_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

submit key simulator_0
submit key simulator_1
submit key simulator_2
submit key simulator_3
submit key simulator_4
submit key simulator_5
submit key simulator_6
submit key simulator_7
submit key simulator_8
submit key simulator_9
submit key simulator_10
submit key simulator_11
submit key simulator_12
submit key simulator_13
submit key simulator_14
submit key simulator_15
submit key simulator_16
submit key simulator_17
submit key simulator_18
submit key simulator_19
submit key simulator_20
submit key simulator_21
submit key simulator_22
submit key simulator_23
submit key simulator_24
submit key simulator_25
submit key simulator_26
submit key simulator_27
submit key simulator_28
submit key simulator_29


In [21]:
print(optimize_result["results"].get_dict())


{'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'energy': [-15.501378988984, -15.254935305147, -20.462837437197, -25.523169410725, -9.56895757884, -26.270628981399, -27.509792342029, -24.601064568918, -12.060879812228, -28.797416882541, -12.35411226286, -21.795109164712, -29.838623160171, -15.256591657518, -19.441267333164, -16.545699380078, -31.689223997706, -21.991670380701, -21.333567220949, -25.764000075402, -22.698719725413, -18.903056830739, -19.82000762202, -18.057699755941, -18.579178407125, -18.833634607649, -17.664326739204, -20.680823897072, -24.172607919465, -19.604560605259]}


In [22]:
all_optimized_structures_node = optimize_result["final_structures"]
len(all_optimized_structures_node.structurecollection.keys())


30

# next run

In [23]:
# 修正すること。
# stat, struc_dict = cryspy_restart.restart(all_initial_structures_node.structurecollection)
ea_id_node = initial_result["ea_id_data"]


In [25]:
# optの対称性の登録
from aiida_cryspy.common import generate_rlst

rslt_data_node_new = generate_rlst(all_initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              ea_id_node, rslt_data_node, stat)

# rslt for new EA
rslt_data_node_new.df

,Gen,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt
0,1,79,I4,1,P1,-1.937672,None,done
1,1,56,Pccn,56,Pccn,-1.906867,None,done
2,1,65,Cmmm,65,Cmmm,-2.557855,None,done
3,1,4,P2_1,4,P2_1,-3.190396,None,done
4,1,67,Cmme,13,P2/c,-1.196120,None,done
5,1,107,I4mm,8,Cm,-3.283829,None,done
6,1,5,C2,1,P1,-3.438724,None,done
7,1,139,I4/mmm,139,I4/mmm,-3.075133,None,done
8,1,13,P2/c,13,P2/c,-1.507610,None,done
9,1,122,I-42d,1,P1,-3.599677,None,done


# next_sg

It generates new candidates by EA.


In [26]:
ea_data_node = initial_result["ea_data"]
rslt_data_node = rslt_data_node_new

inputs = {'initial_structures': all_initial_structures_node,
         'optimized_structures': all_optimized_structures_node,
          'rslt_data': rslt_data_node,
          'ea_id_data': ea_id_node,
          'ea_data': ea_data_node,
         'stat': stat}
next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)



Done generation 1

# ---------- Evolutionary algorithm

# ------ Generation 2
# -- select parents
Remove duplicated data
Survival of the fittest: top 10 structures survive
# -- Generate structures


mindist in _add_border_line: N - N, 1.8206897431990494. retry.
mindist in _add_border_line: N - N, 1.7150083696614755. retry.
mindist in _add_border_line: N - Ga, 0.7614010146774164. retry.
mindist in _add_border_line: Ga - Ga, 1.8765439167206435. retry.
mindist in _add_border_line: Ga - Ga, 1.757672587547272. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
mindist in _add_border_line: Ga - Ga, 0.8274078740803825. retry.
mindist in _add_border_line: Ga - Ga, 1.7294921280962459. retry.
mindist in _add_border_line: Ga - Ga, 1.4338449603823602. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
mindist in permutation: Ga - Ga, 1.9603680403015011. retry.
mindist in permutation: Ga - Ga, 1.9430851042428248. retry.


Structure ID     30 was generated from     16 and     28 by crossover. Space group:   1 P1
Structure ID     31 was generated from      9 and      6 by crossover. Space group:   1 P1
Structure ID     32 was generated from     16 and     12 by crossover. Space group:   1 P1
Structure ID     33 was generated from     19 and      5 by crossover. Space group:   1 P1
Structure ID     34 was generated from     16 and     12 by crossover. Space group:   1 P1
Structure ID     35 was generated from     16 and     28 by crossover. Space group:   1 P1
Structure ID     36 was generated from     12 and     16 by crossover. Space group:   1 P1
Structure ID     37 was generated from     16 and      9 by crossover. Space group:   1 P1
Structure ID     38 was generated from      6 and     16 by crossover. Space group:   1 P1
Structure ID     39 was generated from     12 and     16 by crossover. Space group:   1 P1


mindist in permutation: Ga - Ga, 1.9430851042428248. retry.
mindist in permutation: Ga - Ga, 1.983170821406625. retry.
mindist in permutation: Ga - Ga, 1.9430851042428248. retry.
mindist in permutation: Ga - Ga, 1.983170821406625. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: N - N, 1.934460742732003. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: N - N, 1.934460742732003. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.993620607967883. retry.
m

Structure ID     40 was generated from      5 by permutation. Space group:   8 Cm


mindist in permutation: Ga - Ga, 1.993620607967883. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.9894279255608374. retry.
mindist in permutation: Ga - Ga, 1.9410362416150317. retry.
mindist in permutation: Ga - Ga, 1.9410362416150317. retry.
mindist in permutation: Ga - Ga, 1.9410362416150317. retry.
mindist in permutation: Ga - Ga, 1.9410362416150317. retry.
mindist in permutation: N - N, 1.993620607967883. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: N - N, 1.9894279255608374. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: N - N, 1.981745661297256. retry.
mindist in permutation: N - N, 1.993620607967883. retry.
mindist in permutation: N - N, 1.9894279255608374. retry.
mindist in permutation: N - N, 1.9894279255608374. retry.
mindist 

Permutatin: could not satisfy min_dist in 50 times
Change parent
Structure ID     41 was generated from      3 by permutation. Space group:   1 P1
Structure ID     42 was generated from     19 by permutation. Space group:   1 P1
Structure ID     43 was generated from     16 by strain. Space group:   1 P1
Structure ID     44 was generated from      5 by strain. Space group:   1 P1
Structure ID     45 was generated from     12 by strain. Space group:   2 P-1
Structure ID     46 was generated from     16 by strain. Space group:   1 P1
Structure ID     47 was generated from      6 by strain. Space group:   1 P1
Structure ID     48 was generated. Space group:  48 -->  71 Immm
Structure ID     49 was generated. Space group:  15 -->  15 C2/c
# -- select elites


mindist in permutation: N - N, 1.6392441544872434. retry.
mindist in permutation: N - N, 1.8259318704330247. retry.
mindist in permutation: N - N, 1.077998054210463. retry.
mindist in permutation: N - N, 1.633431200937059. retry.
mindist in permutation: N - N, 1.324005577454683. retry.
mindist in permutation: N - N, 1.354900619173365. retry.
mindist in permutation: N - N, 0.8471049357226338. retry.
mindist in permutation: N - N, 1.595462376186433. retry.
mindist in permutation: N - N, 1.9488796050061397. retry.
mindist in permutation: N - N, 1.5077671662671168. retry.
mindist in permutation: N - N, 1.3249961178502068. retry.
mindist in permutation: N - N, 1.4897749256998103. retry.
mindist in permutation: Ga - Ga, 1.5727443567603792. retry.
mindist in permutation: N - N, 1.0420278149403714. retry.
mindist in permutation: N - N, 1.0833890304403262. retry.
mindist in permutation: N - Ga, 0.939511730274408. retry.
mindist in permutation: Ga - Ga, 0.8666439244292609. retry.
mindist in perm

Remove duplicated data
Survival of the fittest: top 3 structures survive
Structure ID     16 keeps as the elite
Structure ID     12 keeps as the elite
Structure ID      9 keeps as the elite
# -- changed cryspy.in
Changed the value of tot_struc in cryspy.in from 30 to 50


In [27]:
next_sg_results

{'ea_id_data': <EAidData: uuid: f77ca584-806e-49d4-ae5a-34a00dc33d75 (pk: 32508)>,
 'ea_data': <EAData: uuid: 9c759760-14dc-4828-b204-0c8c533e8823 (pk: 32509)>,
 'rslt_data': <PandasFrameData: uuid: 8026755e-65b0-4b8d-90b4-f50be2716013 (pk: 32510)>,
 'initial_structures': <StructureCollectionData: uuid: 8b9c7db2-3464-40ba-b7c9-6a339fbeefb1 (pk: 32511)>,
 'stat': <ConfigParserData: uuid: eb1ccfc8-a385-468c-b696-7aa7685cd3d5 (pk: 32512)>}

In [42]:
stat_node = next_sg_results["stat"]
stat_node.get_dict()

{'EA': {'n_pop': '20',
  'n_perm': '3',
  'n_rand': '2',
  'ntimes': '1',
  't_size': '2',
  'crs_lat': 'equal',
  'emax_ea': 'None',
  'emin_ea': 'None',
  'n_crsov': '10',
  'n_elite': '3',
  'n_strain': '5',
  'sigma_st': '0.5',
  'maxcnt_ea': '50',
  'maxgen_ea': '3',
  'n_fittest': '10',
  'slct_func': 'TNM',
  'fit_reverse': 'False',
  'mindist_ea_1': '2.0 1.0',
  'mindist_ea_2': '1.0 2.0',
  'nat_diff_tole': '4'},
 'basic': {'algo': 'EA',
  'njob': '30',
  'jobcmd': 'bash',
  'nstage': '1',
  'jobfile': 'job_cryspy',
  'calc_code': 'LAMMPS',
  'tot_struc': '50'},
 'LAMMPS': {'lammps_data': 'structure.data',
  'lammps_infile': 'in.lammps',
  'lammps_outfile': 'out.lammps',
  'lammps_potential': 'GaN.tersoff'},
 'option': {'recalc': '',
  'stop_chkpt': '0',
  'append_struc_ea': 'False',
  'force_step_flag': 'False',
  'load_struc_flag': 'False',
  'stop_next_struc': 'False',
  'struc_step_flag': 'False',
  'energy_step_flag': 'False',
  'stress_step_flag': 'False'},
 'status': {'g

In [29]:
ea_id_node = next_sg_results["ea_id_data"]
print(ea_id_node.ea_id_data)


(2, [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [])


In [30]:
all_initial_structures_node = next_sg_results["initial_structures"]
all_initial_structures_node.structurecollection.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [31]:
next_sg_results["rslt_data"].df.sort_values(by="E_eV_atom")

,Gen,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt
16,1,156,P3m1,156,P3m1,-3.961153,None,done
12,1,225,Fm-3m,225,Fm-3m,-3.729828,None,done
9,1,122,I-42d,1,P1,-3.599677,None,done
6,1,5,C2,1,P1,-3.438724,None,done
5,1,107,I4mm,8,Cm,-3.283829,None,done
19,1,71,Immm,2,P-1,-3.220500,None,done
3,1,4,P2_1,4,P2_1,-3.190396,None,done
7,1,139,I4/mmm,139,I4/mmm,-3.075133,None,done
28,1,53,Pmna,53,Pmna,-3.021576,None,done
20,1,74,Imma,2,P-1,-2.837340,None,done


In [32]:
all_initial_structures_node.structurecollection.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

# next structure  optimization

In [33]:
inputs = {"ea_id_data":  ea_id_node, "init_struc": all_initial_structures_node }

select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["init_struc"]
print(work_path_node.get_dict().keys(),  structures_to_optimize_node.get_dict().keys())

dict_keys(['30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49']) dict_keys(['30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49'])


In [34]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)

submit key simulator_30
submit key simulator_31
submit key simulator_32
submit key simulator_33
submit key simulator_34
submit key simulator_35
submit key simulator_36
submit key simulator_37
submit key simulator_38
submit key simulator_39
submit key simulator_40
submit key simulator_41
submit key simulator_42
submit key simulator_43
submit key simulator_44
submit key simulator_45
submit key simulator_46
submit key simulator_47
submit key simulator_48
submit key simulator_49


In [35]:
print(optimize_result["results"].get_dict())

{'index': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'energy': [-15.215088132507, -30.107087165897, -22.571369581776, -31.054330699164, -28.970559231963, -15.957625356516, -27.407378210242, -29.919745857421, -26.201621321945, -28.320539170165, -27.041086649307, -26.478034535986, -25.226879910246, -32.371063238327, -26.788111118068, -19.70165596737, -7.2431664309212, -15.563316288962, -14.999490929556, -25.932337257133]}


In [36]:
# merge all_optimized_structures_node
all_optimized_structures_node.structurecollection.keys(), optimize_result["final_structures"].structurecollection.keys()

(dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 dict_keys([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]))

In [37]:
import copy
@calcfunction
def merge_structurecollection(struc1: StructurecollectionData, struc2: StructurecollectionData):
    struc1 = struc1.structurecollection
    struc2 = struc2.structurecollection
    for key, value in struc2.items():
        if key not in struc1:
            struc1[key] = value
        else:
            print(f"Warning: key {key} is in struc1.")
    return StructurecollectionData(struc1)

In [38]:
all_optimized_structures_node = merge_structurecollection(all_optimized_structures_node, optimize_result["final_structures"])
all_optimized_structures_node.structurecollection.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [39]:
ea_id_node = next_sg_results["ea_id_data"]
all_initial_structures_node = next_sg_results["initial_structures"]
all_initial_structures_node.structurecollection.keys(), all_optimized_structures_node.structurecollection.keys()

(dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]))

In [44]:
rslt_data_node = generate_rlst(all_initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              ea_id_node, rslt_data_node, stat_node)
rslt_data_node.df.sort_values(by="E_eV_atom")

,Gen,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt
43,2,1,P1,1,P1,-4.046383,None,done
16,1,156,P3m1,156,P3m1,-3.961153,None,done
33,2,1,P1,1,P1,-3.881791,None,done
31,2,1,P1,1,P1,-3.763386,None,done
37,2,1,P1,1,P1,-3.739968,None,done
12,1,225,Fm-3m,225,Fm-3m,-3.729828,None,done
34,2,1,P1,1,P1,-3.621320,None,done
9,1,122,I-42d,1,P1,-3.599677,None,done
39,2,1,P1,1,P1,-3.540067,None,done
6,1,5,C2,1,P1,-3.438724,None,done


In [45]:
ea_data_node = next_sg_results["ea_data"]
ea_id_data_node = select_structure_result["ea_id_data"]


In [46]:
print(ea_id_data_node.get_dict())


{'gen': 2, 'id_running': [], 'id_queueing': []}


In [47]:
all_optimized_structures_node.structurecollection.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

# next_sg

In [48]:
rslt_data_node.df.sort_values(by='E_eV_atom')

,Gen,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt
43,2,1,P1,1,P1,-4.046383,None,done
16,1,156,P3m1,156,P3m1,-3.961153,None,done
33,2,1,P1,1,P1,-3.881791,None,done
31,2,1,P1,1,P1,-3.763386,None,done
37,2,1,P1,1,P1,-3.739968,None,done
12,1,225,Fm-3m,225,Fm-3m,-3.729828,None,done
34,2,1,P1,1,P1,-3.621320,None,done
9,1,122,I-42d,1,P1,-3.599677,None,done
39,2,1,P1,1,P1,-3.540067,None,done
6,1,5,C2,1,P1,-3.438724,None,done


In [49]:
stat["basic"]

{'algo': 'EA',
 'njob': '30',
 'jobcmd': 'bash',
 'nstage': '1',
 'jobfile': 'job_cryspy',
 'calc_code': 'LAMMPS',
 'tot_struc': '50'}

In [50]:
inputs = {'initial_structures': all_initial_structures_node,
         'optimized_structures': all_optimized_structures_node,
          'rslt_data': rslt_data_node,
          'ea_id_data': ea_id_data_node,
          'ea_data': ea_data_node,
         'stat': stat}

next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)



Done generation 2

# ---------- Evolutionary algorithm

# ------ Generation 3
# -- select parents
Remove duplicated data
Survival of the fittest: top 10 structures survive
# -- Generate structures
Structure ID     50 was generated from     37 and     43 by crossover. Space group:   1 P1


some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
mindist in _add_border_line: N - N, 1.5577719686804956. retry.
mindist in _add_border_line: N - N, 1.945963689057542. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
mindist in _add_border_line: N - N, 1.9155244591607088. retry.
mindist in _add_border_line: N - Ga, 0.3872010609079648. retry.
mindist in _add_border_line: N - N, 1.8749100035099646. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some 

Structure ID     51 was generated from     31 and     33 by crossover. Space group:   1 P1
Structure ID     52 was generated from     16 and     43 by crossover. Space group:   1 P1
Structure ID     53 was generated from     43 and      9 by crossover. Space group:   1 P1
Structure ID     54 was generated from     37 and     12 by crossover. Space group:   1 P1


some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
some atoms within mindist. retry.
mindist in _add_border_line: N - N, 1.9170582539053596. retry.
some atoms within mindist. retry.
mindist in _add_border_line: N - N, 1.8678119143504541. retry.
mindist in permutation: Ga - Ga, 1.9310663086999789. retry.
mindist in permutation: Ga - Ga, 1.9310663086999789. retry.
mindist in permutation: Ga - Ga, 1.931066308799919. retry.
mindist in permutation: Ga - Ga, 1.9310663086999789. retry.
mindist in permutation: N - N, 1.9168743570854914. retry.
mindist in permutation: N - N, 1.9168743570854914. retry.
mindist in permutation: N - N, 1.909969345899642. retry.
mindist in permutation: N - N, 1.909969345899642. retry.
mindist in permutation: N - N, 1.9168743571175924. retry.
mindist

Structure ID     55 was generated from     31 and     33 by crossover. Space group:   1 P1
Structure ID     56 was generated from     43 and     37 by crossover. Space group:   1 P1
Structure ID     57 was generated from     31 and     34 by crossover. Space group:   1 P1
Structure ID     58 was generated from     31 and      9 by crossover. Space group:   1 P1
Structure ID     59 was generated from      9 and     33 by crossover. Space group:   1 P1
Structure ID     60 was generated from     16 by permutation. Space group: 156 P3m1
Structure ID     61 was generated from     16 by permutation. Space group: 156 P3m1


mindist in permutation: Ga - Ga, 1.9168743570854914. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
mindist in permutation: N - N, 1.9702249885653844. retry.
mindist in permutation: Ga - Ga, 1.9168743570854914. retry.
mindist in permutation: N - N, 1.909969345899642. retry.
mindist in permutation: N - N, 1.909969345899642. retry.
mindist in permutation: N - N, 1.9168743570854914. retry.
mindist in permutation: Ga - Ga, 1.9702249885662546. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
mindist in permutation: N - N, 1.9168743570854914. retry.
mindist in permutation: N - N, 1.9702249885653844. retry.
mindist in permutation: N - N, 1.9702249885653844. retry.
mindist in permutation: Ga - Ga, 1.909969345899642. retry.
min

Permutatin: could not satisfy min_dist in 50 times
Change parent
Structure ID     62 was generated from     34 by permutation. Space group:   1 P1
Structure ID     63 was generated from     37 by strain. Space group:   1 P1
Structure ID     64 was generated from     31 by strain. Space group:   1 P1
Structure ID     65 was generated from     43 by strain. Space group:   1 P1
Structure ID     66 was generated from     37 by strain. Space group:   1 P1
Structure ID     67 was generated from     12 by strain. Space group:  10 P2/m


mindist in permutation: Ga - Ga, 1.7069381868026348. retry.
mindist in permutation: Ga - Ga, 0.5230468288203686. retry.


Structure ID     68 was generated. Space group:  20 -->  20 C222_1
Structure ID     69 was generated. Space group: 205 --> 225 Fm-3m
# -- select elites
Remove duplicated data
Survival of the fittest: top 3 structures survive
Structure ID     43 keeps as the elite
Structure ID     16 keeps as the elite
Structure ID     33 keeps as the elite
# -- changed cryspy.in
Changed the value of tot_struc in cryspy.in from 50 to 70


In [51]:
next_sg_results

{'ea_id_data': <EAidData: uuid: 347f01aa-1c34-4320-a7da-6e42a7adcb2a (pk: 32671)>,
 'ea_data': <EAData: uuid: 35e7bcac-2d01-4ded-820b-2de9adc5d3e1 (pk: 32672)>,
 'rslt_data': <PandasFrameData: uuid: 2c1acd0c-e85a-4300-9ead-b3078ba68994 (pk: 32673)>,
 'initial_structures': <StructureCollectionData: uuid: e2544730-7bfd-4dca-8972-a773d431bd39 (pk: 32674)>,
 'stat': <ConfigParserData: uuid: df65e5f0-d573-4a9a-ba97-94c7e6f56d7d (pk: 32675)>}

In [52]:
ea_id_data = next_sg_results['ea_id_data']


In [53]:
all_initial_structures_node = next_sg_results["initial_structures"]
print(all_initial_structures_node.get_dict().keys())


dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69'])


# structure optimization 

In [54]:
inputs = {"ea_id_data":  ea_id_data, "init_struc": all_initial_structures_node }

select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["init_struc"]


In [55]:
work_path_node.get_dict(), structures_to_optimize_node.get_dict().keys()


({'50': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000050',
  '51': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000051',
  '52': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000052',
  '53': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000053',
  '54': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000054',
  '55': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000055',
  '56': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000056',
  '57': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000057',
  '58': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000058',
  '59': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000059',
  '60': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000060',
  '61': '/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA/work/000061',
  '62': '/home/max/Documents/aiida_crysp

In [56]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

submit key simulator_50
submit key simulator_51
submit key simulator_52
submit key simulator_53
submit key simulator_54
submit key simulator_55
submit key simulator_56
submit key simulator_57
submit key simulator_58
submit key simulator_59
submit key simulator_60
submit key simulator_61
submit key simulator_62
submit key simulator_63
submit key simulator_64
submit key simulator_65
submit key simulator_66
submit key simulator_67
submit key simulator_68
submit key simulator_69


In [57]:
optimize_result

{'results': <Dict: uuid: 255a726f-85d3-48d5-b561-ec0595a82960 (pk: 32824)>,
 'final_structures': <StructureCollectionData: uuid: 2d80d7b1-c2ba-4cf5-9214-6fcc65ad754d (pk: 32826)>}

In [58]:
all_optimized_structures_node = merge_structurecollection(all_optimized_structures_node, optimize_result["final_structures"])
all_optimized_structures_node.structurecollection.keys()


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69])

In [59]:
ea_id_node = next_sg_results["ea_id_data"]
all_initial_structures_node = next_sg_results["initial_structures"]

all_initial_structures_node.structurecollection.keys(), all_optimized_structures_node.structurecollection.keys()


(dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]),
 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]))

In [61]:
rslt_data_node_new = generate_rlst(all_initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              ea_id_node, rslt_data_node, stat_node)
print(rslt_data_node_new.df.shape)
rslt_data_node_new.df.sort_values(by="E_eV_atom").head(50)

(70, 8)


,Gen,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt
57,3,1,P1,1,P1,-4.217786,None,done
56,3,1,P1,1,P1,-4.050593,None,done
43,2,1,P1,1,P1,-4.046383,None,done
16,1,156,P3m1,156,P3m1,-3.961153,None,done
33,2,1,P1,1,P1,-3.881791,None,done
31,2,1,P1,1,P1,-3.763386,None,done
37,2,1,P1,1,P1,-3.739968,None,done
69,3,225,Fm-3m,225,Fm-3m,-3.729828,None,done
12,1,225,Fm-3m,225,Fm-3m,-3.729828,None,done
51,3,1,P1,1,P1,-3.641948,None,done


In [63]:
!verdi node graph generate $rslt_data_node_new.pk

Info: Initiating graphviz engine: dot
Info: Recursing ancestors, max depth=None
Info: Recursing descendants, max depth=None
Success: Output file: 32830.dot.pdf
